In [3]:
from custom_ml import *
import gen as g
from sklearn.linear_model import LogisticRegression
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale

In [4]:
def top_merge(df, n=10, column="predict", merge_column="COUPON_ID_hash"):
    '''
    get top n row

    :param pandas.DataFrame df:
    :param int n:
    :param str column:
    :rtype: pandas.DataFrame
    '''

    return " ".join(df.sort_index(by=column)[-n:][merge_column])

In [5]:
df = g.user_view_coupon_pref()

In [6]:
df.head()

,REG_DATE,SEX_ID,AGE,WITHDRAW_DATE,PREF_NAME_x,USER_ID_hash,PURCHASE_FLG,I_DATE,PAGE_SERIAL,REFERRER_hash,...,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name,COUPON_ID_hash,PREF_NAME_y,PREFECTUAL_OFFICE,LATITUDE,LONGITUDE
0,2012-03-28 14:14:18,f,25,NaN,NaN,d9dca3cb44bab12ba313eaa681f663eb,0,2012-03-28 14:15:00,7,7d3892e54acb559ae36c459978489330,...,NaN,NaN,Kanto,Tokyo,"Shinjuku, Takadanobaba Nakano - Kichijoji",34c48f84026e08355dc3bd19b427f09a,Tokyo,Shinjuku,35.689521,139.691704
1,2012-03-28 14:14:18,f,25,NaN,NaN,d9dca3cb44bab12ba313eaa681f663eb,0,2012-03-28 14:17:28,9,7d3892e54acb559ae36c459978489330,...,NaN,NaN,Kanto,Tokyo,"Shinjuku, Takadanobaba Nakano - Kichijoji",34c48f84026e08355dc3bd19b427f09a,Tokyo,Shinjuku,35.689521,139.691704
2,2012-03-28 14:14:18,f,25,NaN,NaN,d9dca3cb44bab12ba313eaa681f663eb,0,2012-03-28 14:32:02,25,7d3892e54acb559ae36c459978489330,...,NaN,NaN,Kanto,Tokyo,"Shinjuku, Takadanobaba Nakano - Kichijoji",34c48f84026e08355dc3bd19b427f09a,Tokyo,Shinjuku,35.689521,139.691704
3,2012-03-28 14:14:18,f,25,NaN,NaN,d9dca3cb44bab12ba313eaa681f663eb,1,2012-03-28 15:06:11,45,878034ded0ee2297d7ce27c5169e8584,...,NaN,NaN,Kanto,Tokyo,"Shinjuku, Takadanobaba Nakano - Kichijoji",34c48f84026e08355dc3bd19b427f09a,Tokyo,Shinjuku,35.689521,139.691704
4,2012-03-28 14:14:18,f,25,NaN,NaN,d9dca3cb44bab12ba313eaa681f663eb,0,2012-03-28 15:08:10,53,7d3892e54acb559ae36c459978489330,...,NaN,NaN,Kanto,Tokyo,"Shinjuku, Takadanobaba Nakano - Kichijoji",34c48f84026e08355dc3bd19b427f09a,Tokyo,Shinjuku,35.689521,139.691704


In [7]:
df.columns

Index(['REG_DATE', 'SEX_ID', 'AGE', 'WITHDRAW_DATE', 'PREF_NAME_x',
       'USER_ID_hash', 'PURCHASE_FLG', 'I_DATE', 'PAGE_SERIAL',
       'REFERRER_hash', 'VIEW_COUPON_ID_hash', 'SESSION_ID_hash',
       'PURCHASEID_hash', 'CAPSULE_TEXT', 'GENRE_NAME', 'PRICE_RATE',
       'CATALOG_PRICE', 'DISCOUNT_PRICE', 'DISPFROM', 'DISPEND', 'DISPPERIOD',
       'VALIDFROM', 'VALIDEND', 'VALIDPERIOD', 'USABLE_DATE_MON',
       'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU',
       'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN',
       'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name',
       'ken_name', 'small_area_name', 'COUPON_ID_hash', 'PREF_NAME_y',
       'PREFECTUAL_OFFICE', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

# 1. 아웃라이어 제거 
(하려고 했지만 로지스틱 리그레션은 아웃라이어에 robust하단다)
# 2. 스케일링
# 3. 정규화
# 4. 

# SEX_ID, AGE, ITEM_COUNT, GENRE_NAME, PRICE_RATE, CATALOG_PRICE, DISCOUNT_PRICE, DISPERIOD, VALIDPERIOD, ken_name, LATITUDE, LONGITUDE, PAGE_SERIAL

In [10]:
formula = 'PURCHASE_FLG ~ C(GENRE_NAME) + DISCOUNT_PRICE + C(ken_name)'

In [11]:
model = sm.Logit.from_formula(formula, df)
result = model.fit(disp=0)

In [ ]:
# predict test data
predict_proba = clf.predict_proba(X_test)

# 클래스가 1이면 산다라고 분류
# 그러니까 아래 코드는 사는 놈들의 스코어를 위한 인덱스
pos_idx = np.where(clf.classes_ == True)[0][0]

test_df["predict"] = predict_proba[:, pos_idx]
top10_coupon = test_df.groupby("USER_ID_hash").apply(top_merge)
top10_coupon.name = "PURCHASED_COUPONS"
top10_coupon.to_csv("submission.csv", header=True)